In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
os.environ['PATH']+= r'C:/SeleniumDrivers'

In [ ]:
driver=webdriver.Chrome()
driver.get('https://www.bigbasket.com/pc/beverages/energy-soft-drinks/?nc=nb')

In [ ]:
elements=driver.find_elements(By.CSS_SELECTOR,'a.h-full')

In [ ]:
for i,j in enumerate(elements):
    print(j.text)
    print(i)
    print("........................")

In [ ]:
data_list = []

def individual():
    try:
        prod_name = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'Description___StyledH-sc-82a36a-2'))).text
    except (NoSuchElementException, TimeoutException):
        prod_name = None

    try:
        price = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'Description___StyledTd-sc-82a36a-4'))).text
    except (NoSuchElementException, TimeoutException):
        price = None

    try:
        ingredients_label = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//span[text()="Ingredients"]')))
        ingredients_div = ingredients_label.find_element(By.XPATH, '../following-sibling::div[@class="overflow-hidden"]')
        ingredients_text = ingredients_div.text
    except (NoSuchElementException, TimeoutException):
        ingredients_text = None

    # Append the data to the list
    print(prod_name, price, ingredients_text)
    data_list.append({'Product Name': prod_name, 'Price': price, 'Ingredients': ingredients_text})


In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException, TimeoutException, StaleElementReferenceException
import time

i = 0

for element in elements:
    try:
        # Click on the element directly
        for retry in range(3):  # Adjust the number of retries as needed
            try:
                driver.implicitly_wait(5)
                element.click()
                break
            except StaleElementReferenceException:
                print("StaleElementReferenceException occurred. Retrying...")
                continue

        print("Clicked on the element")

        # Wait for the specific element to become invisible
        WebDriverWait(driver, 30).until(
            EC.invisibility_of_element_located((By.XPATH, '//a[@class="h-full"][@href="/pd/40276397/indisecrets-assorted-tulsi-infusions-with-4-exotic-flavours-sampler-pack-85-g/?nc=cl-prod-list&amp;t_pos_sec=3&amp;t_pos_item=3&amp;t_s=Assorted+Tulsi+Infusions+With+4+Exotic+Flavours"]'))
        )
        print("Element became invisible")

        # Wait for a new window to open
        WebDriverWait(driver, 30).until(lambda driver: len(driver.window_handles) > 1)
        print("New window opened")

        # Switch to the new window
        driver.switch_to.window(driver.window_handles[1])
        print("Switched to the new window")

        # Perform actions in the new window (assuming 'individual()' is defined elsewhere)
        individual()
        WebDriverWait(driver, 30).until(lambda driver: len(driver.window_handles) > 1)  # Wait for any additional pop-ups if needed
        time.sleep(2)

        # Close the new window
        driver.close()
        print("Closed the new window")

        # Switch back to the original window
        driver.switch_to.window(driver.window_handles[0])
        print("Switched back to the original window")

    except (WebDriverException, TimeoutException, StaleElementReferenceException) as e:
        print(f"Error occurred: {e}")
        continue


In [ ]:
# Create the DataFrame after the loop
df = pd.DataFrame(data_list)

# Print or use the DataFrame as needed
print(df)


In [ ]:
excel_filename = 'output.xlsx'
df.to_excel(excel_filename, index=False)